In [1]:
import pandas as pd
import numpy as np
import datetime
from pandas.tseries.offsets import DateOffset
from tqdm import tqdm

<b>Загружаем словарь из файла

In [2]:
# file_sasha = ''
file_vanya = 'dictionary_compressed.csv'
file_phil = 'D:\Documents\VISA_course\Final Round\dictionary_compressed.csv'
dictionary = pd.read_csv(file_phil,sep='\t').to_dict('records')[0]


<b>Считывание файла из папочки, и использование менее затратных типов данных по памяти.

In [3]:
# file_sasha2 = ''
file_vanya2 = 'compressed_data_win.csv'
file_phil2 = 'D:\Documents\VISA_course\Final Round\compressed_data_win.csv'
datad_start =  pd.read_csv(file_phil2,sep='\t', dtype ={
    'purchase_dt':'category',
    
    'sample_data':'uint8','acct_num_share':'uint32', 
'issr_code_numb':'uint8','product_nm':'uint16','funding_source':'uint16','mrch_country_nm':'uint32', 
'f2f_ecomm_flg':'uint16','pos_cash_flg':'uint16','mrch_category_01':'uint16','mrch_category_02':'uint16', 
'mrch_category_03':'uint16','mrch_category_04':'uint16',

    'rub_amt':'float32'   
    })

# Алгоритм сжатия: 14 гб --> 2,3 гб

In [4]:
print(datad_start.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78498158 entries, 0 to 78498157
Data columns (total 14 columns):
 #   Column            Dtype   
---  ------            -----   
 0   sample_data       uint8   
 1   acct_num_share    uint32  
 2   issr_code_numb    uint8   
 3   product_nm        uint16  
 4   funding_source    uint16  
 5   mrch_country_nm   uint32  
 6   f2f_ecomm_flg     uint16  
 7   pos_cash_flg      uint16  
 8   rub_amt           float32 
 9   purchase_dt       category
 10  mrch_category_01  uint16  
 11  mrch_category_02  uint16  
 12  mrch_category_03  uint16  
 13  mrch_category_04  uint16  
dtypes: category(1), float32(1), uint16(8), uint32(2), uint8(2)
memory usage: 2.3 GB
None


##  Случайно выбираем подсет из 10000 карт.

In [5]:
# number_of_cards = 10000

# a = np.random.choice(datad_start['acct_num_share'].unique(),size = number_of_cards,replace = False) 

# datad = datad_start[datad_start['acct_num_share'].isin(a)]

Посчитаем на  всех картах

In [6]:
datad = datad_start

In [7]:
datad['purchase_dt'] = datad['purchase_dt'].astype('datetime64[ns]')

In [8]:
last_day = datetime.date(2018, 9, 30)
time_point = last_day - pd.offsets.MonthEnd(1)
print(time_point)

2018-08-31 00:00:00


In [9]:
time_start = datetime.datetime.utcnow()

## Строим фичи


In [10]:
cards = datad['acct_num_share'] 
indd = np.unique(cards)
first_date = datetime.date(2016, 10, 1)
first_date = last_day - DateOffset(months=3) 

pbar = tqdm(total=81)

for mth in range(0, 12):
    
    ft = pd.DataFrame(np.zeros((indd.shape[0],342)), index=indd)
    time_point = first_date - DateOffset(months=mth)
    
    time_full = time_point - datad.purchase_dt.min()
    time_full = int(str(time_full).split()[0])
    
    s = 0
    w_ins = [1, 2, 3]
    for j in w_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt'])])['rub_amt'].mean(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    

    pbar.update(1)

    m_ins = [1, 3, 6, 9, 12]
    for j in m_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt'])])['rub_amt'].mean(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    
    for j in w_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt']) &
         (x['mrch_category_03'] == dictionary['TRAVEL'])])['rub_amt'].mean(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    for j in m_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt']) &
         (x['mrch_category_03'] == dictionary['TRAVEL'])])['rub_amt'].mean(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)

    for j in w_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt']) &
         (x['mrch_category_03'] == dictionary['TRAVEL'])])['rub_amt'].std(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    for j in m_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt']) &
         (x['mrch_category_03'] == dictionary['TRAVEL'])])['rub_amt'].std(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)

    for j in w_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt'])])['rub_amt'].std(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    for j in m_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt'])])['rub_amt'].std(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)

    for j in w_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt']) &
         (x['pos_cash_flg'] == dictionary['CASH'])])['rub_amt'].count(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    for j in m_ins:
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt']) &
         (x['pos_cash_flg'] == dictionary['CASH'])])['rub_amt'].count(level='acct_num_share')
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)

    s = 52

    for j in w_ins:
        mean1 =(datad.groupby('acct_num_share').count()['sample_data']/time_full) * 7.0 * j
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt'])])['rub_amt'].count(level='acct_num_share')/mean1
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    for j in m_ins:
        mean1 =(datad.groupby('acct_num_share').count()['sample_data']/time_full) * 30.0 * j
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt'])])['rub_amt'].count(level='acct_num_share')/mean1
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)

    s = 80
    m_ins = [1, 2, 3]

    for j in w_ins:
        mean1 =(datad.groupby('acct_num_share').count()['sample_data']/time_full) * 7.0 * j
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt']) &
         (x['mrch_category_02'] == dictionary['FUEL/SERVICE STATION'])])['rub_amt'].count(level='acct_num_share')/mean1
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    for j in m_ins:
        mean1 =(datad.groupby('acct_num_share').count()['sample_data']/time_full) * 7.0 * j
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt']) &
         (x['mrch_category_02'] == dictionary['FUEL/SERVICE STATION'])])['rub_amt'].count(level='acct_num_share')/mean1
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)

    s = 134
    m_ins = [1, 3]
    w_ins = [1, 2]

    for j in w_ins:
        mean1 =(datad.groupby('acct_num_share').count()['sample_data']/time_full) * 7.0 * j
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - pd.offsets.Week(j)) < x['purchase_dt']) &
         (x['pos_cash_flg'] == dictionary['CASH'])])['rub_amt'].count(level='acct_num_share')/mean1
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)
    for j in m_ins:
        mean1 =(datad.groupby('acct_num_share').count()['sample_data']/time_full) * 7.0 * j
        ft.iloc[:,s] = datad.groupby('acct_num_share').apply(lambda x: x[(x['purchase_dt']<time_point) &
         ((time_point - DateOffset(months=j)) < x['purchase_dt']) &
         (x['pos_cash_flg'] == dictionary['CASH'])])['rub_amt'].count(level='acct_num_share')/mean1
        ft[s] = ft[s].fillna(0)
        s = s + 1
    pbar.update(1)

#     s = 138
#     x = [5000, 10000, 20000, 30000, 50000]

#     for j in x:
#         for i in indd:
#             k = datad[datad['mrch_category_02'] == dictionary['RESTAURANTS']]
#             k = k[k['acct_num_share'] == i ] 
#             mean1 = (k['rub_amt'].sum()/time_full) * 30.0 
#             if mean1 > j:
#                 ft.loc[i,s] = 1
#             else:
#                 ft.loc[i,s] = 0
#         ft[s] = ft[s].fillna(0)
#         s = s + 1
#     pbar.update(1)
    
 ###############################################################
        
##############
#ФИЧИ ФИЛИППА#
##############
        

    #Писал PHILIPP
    #Фичи из столбцов 40, 41, 42, 43
    #Сумма покупок F2F за последние 3 месяцев, 6, 9, 12
    a0 = 40
    a1 = 43

    m = 0

    for j in [3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])]

        t = k.groupby('acct_num_share').count()['rub_amt']

        ft.loc[:,a0+m] = t
        m=m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Фичи из столбцов 44, ..., 51
    #Превышение среднего значения потраченных денег
    #по данному типу карты у данного клиента
    #превышает  среднее значение по типу карты
    a0 = 44
    a1 = 51


    m = 0

    for j in [1,2,3]:


        k = datad[(datad['purchase_dt']<time_point)&
                    ( time_point - DateOffset(weeks=j) <datad['purchase_dt'])&
                    (datad['funding_source'] == dictionary['D'])]

        t1 = k.groupby('acct_num_share').mean()['rub_amt']  - k['rub_amt'].mean()  

        ft.loc[ : ,a0+m] = t1


        k = datad[(datad['purchase_dt']<time_point)&
                    ( time_point - DateOffset(weeks=j) <datad['purchase_dt'])&
                    (datad['funding_source'] == dictionary['C'])]

        t2 = k.groupby('acct_num_share').mean()['rub_amt']  - k['rub_amt'].mean()  

        ft.loc[ t2.index ,a0+m] = t2 

        m = m+1



    for j in [1,3,6,9,12]:
        k = datad[(datad['purchase_dt']<time_point)&
                    ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                    (datad['funding_source'] == dictionary['D'])]

        t1 = k.groupby('acct_num_share').mean()['rub_amt']  - k['rub_amt'].mean()  

        ft.loc[ : ,a0+m] = t1


        k = datad[(datad['purchase_dt']<time_point)&
                    ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                    (datad['funding_source'] == dictionary['C'])]

        t2 = k.groupby('acct_num_share').mean()['rub_amt']  - k['rub_amt'].mean()  

        ft.loc[ t2.index ,a0+m] = t2 

        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)    

    pbar.update(1)
    #Ускорено
    
    #Писал PHILIPP
    #Фичи из столбцов 60, ..., 64
        #Количество трат выше Х за последний год , Х = ...

    a0=60
    a1=64
    m=0

    for j in [10000, 20000, 30000, 50000, 100000]:
        k = datad[(datad['purchase_dt']<time_point)&( time_point - DateOffset(years=1) <datad['purchase_dt'])&(datad['rub_amt'] > j)]
        t = k.groupby('acct_num_share').count()['rub_amt']

        ft.loc[:, a0+m] = t
        m=m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)


    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Фичи из столбцов 65, ..., 70
    #Среднее колчество потраченных денег FASHION RETAIL за 

    a0=65
    a1=70
    m = 0

    for j in [1, 2, 3]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(weeks = j) <datad['purchase_dt'])&
                  (datad['mrch_category_02']  == dictionary['FASHION RETAIL'] )
                 ]

        a =  k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for j in [1, 2, 3]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months = j) <datad['purchase_dt'])&
                  (datad['mrch_category_02']  == dictionary['FASHION RETAIL'] )
                 ]

        a =  k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Появление затар на обучение заграницей 6 мес, 12 мес
    #Столбцы номер 71, 72

    a0 = 71
    a1 = 72
    m=0


    for j in [6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['EDUCATION'])&
                  (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])
                 ]
        t = k.groupby('acct_num_share').count()['product_nm']
        t[t>0] = 1

        ft.loc[:,a0+m] = t
        m=m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)    

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    # Появление затрат на  развлечение
    # SPORTING/RECREATIONAL CAMPS
    # оплата за границей

    # столбцы 73,.., 76

    a0 = 73
    a1 = 76
    m = 0

    for j in [2]:
        k = datad[
                    (datad['purchase_dt']<time_point)&
                    (time_point - DateOffset(weeks=j) <datad['purchase_dt'])&
                    (datad['mrch_category_01'] == dictionary['SPORTING/RECREATIONAL CAMPS'])&
                    (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])
                 ]

        t = k.groupby('acct_num_share').count()['product_nm']
        t[t>0] = 1

        ft.loc[:,a0+m] = t
        m=m+1


    for j in [1, 2, 3]:
        k = datad[
                    (datad['purchase_dt']<time_point)&
                    (time_point - DateOffset(months=j) <datad['purchase_dt'])&
                    (datad['mrch_category_01'] == dictionary['SPORTING/RECREATIONAL CAMPS'])&
                    (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])
                 ]

        t = k.groupby('acct_num_share').count()['product_nm']
        t[t>0] = 1

        ft.loc[:,a0+m] = t
        m=m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #MEMBER CLUBS/SPORT/REC/GOLF
    #оплата за границей
    #столбцы номер 77, .., 79


    m = 0
    a0 = 77
    a1 = 79

    for j in [1,2,3]:
        k = datad[ (datad['purchase_dt']<time_point)&
                   ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_01'] == dictionary['MEMBER CLUBS/SPORT/REC/GOLF'])&
                  (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])
                 ]

        t = k.groupby('acct_num_share').count()['product_nm']
        t[t>0] = 1

        ft.loc[:,a0+m] = t
        m=m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 
    #Код работает верно  

    #Ускорено
    pbar.update(1)
    #Писал PHILIPP
    #Траты за последние 1,3,6,12 месяцев 
    # превышают 50, 100, 150, 300, 500, 1000 тыс.
    #за 1, 3, 6, 12 мес
    #столбцы номер 86, .. , 109



    a0 = 86
    a1 = 109
    ts = 10**3
    m = 0


    for j in [1, 3, 6, 12]:
        for am in [50*ts, 100*ts, 150*ts, 300*ts, 500*ts, 1000*ts]:
            k = datad[
                      (datad['purchase_dt']<time_point)&
                      ( time_point - DateOffset(months=j) <datad['purchase_dt'])]

            a = k.groupby('acct_num_share').sum()['rub_amt']
            a[a>am] = 1
            a[a!=1]=0

            ft.loc[:,a0+m] = a
            m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Улучшено

    #Писал PHILIPP
    #Наличие F2F покупок за гарницей за последние 
    #3,6,9 mes
    # Ячейки номер 110-112

    a0 = 110
    a1 = 112
    m = 0


    for j in [ 3, 6, 12]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])&
                  (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])
                 ]

        t = k.groupby('acct_num_share').count()['rub_amt']

        t[t>0] = 1
        t[t!=1] = 0

        ft.loc[:,a0+m] = t
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Наличие покупок в LUXURY GOODS за последние 3. 6.  12 mes
    #столбцы номер 113, .. , 115

    a0 = 113
    a1 = 115

    m = 0
    for j in [ 3, 6, 12]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                      ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                      (datad['mrch_category_02'] == dictionary['LUXURY GOODS RETAIL'])
                     ]

        t = k.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0
        ft.loc[:,a0+m] = t
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Наличие покупок в SPA/BEAUTY  за посл 1,3,6,12 мес
    #столбцы номер 116, .. , 119

    a0 = 116
    a1 = 119

    m = 0
    for j in [1, 3, 6, 12]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                      ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                      (datad['mrch_category_02'] == dictionary['SPA / BEAUTY SERVICES'])
                     ]

        t = k.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0
        ft.loc[:,a0+m] = t
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)


    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Наличие покупок в AIRLINES за посл   за посл 
    # 2 нед,1,2,3,6мес
    #столбцы номер 120, .. , 124

    a0 = 120
    a1 = 124


    m = 0
    for j in [2]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                      ( time_point - DateOffset(weeks=j) <datad['purchase_dt'])&
                      (datad['mrch_category_02'] == dictionary['AIRLINES'])
                     ]

        t = k.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0
        ft.loc[:,a0+m] = t
        m = m+1


    for j in [1,2, 3, 6]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                      ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                      (datad['mrch_category_02'] == dictionary['AIRLINES'])
                     ]

        t = k.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0
        ft.loc[:,a0+m] = t
        m = m+1



    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    # Тип обслуживаемой карты типа карты 0/1
    #Столбцы номер 125, ..., 131

    card_types = [3349, 3350, 3351, 3352, 3353, 3354, 3355]


    a0 = 125
    a1 = 131

    m=0


    for j in card_types: 
        t = datad[datad['product_nm'] == j]
        t = t.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0

        ft.loc[:,a0+m] = t
        m = m+1


    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    # Тип карты D - 0/1
    # C - 0/1
    # Столбцы номер 132, 133

    # dictionary['D'] = 3356
    # dictionary['C'] = 3357
    a0 = 132
    a1 = 133

    m = 0

    card_type_dc = [3356, 3357]

    for j in card_type_dc: 
        t = datad[datad['funding_source'] == j]
        t = t.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0

        ft.loc[:,a0+m] = t
        m = m+1


    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Наличие трат в TRAVEL AGENCIES за последние
    # 2 нед
    # 1м
    # 3м
    # 6м
    # Столбцы номер 143, 146

    a0 = 143
    a1 = 146

    m = 0
    for j in [2]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                      ( time_point - DateOffset(weeks=j) <datad['purchase_dt'])&
                      (datad['mrch_category_02'] == dictionary['TRAVEL AGENCIES'])
                     ]

        t = k.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0
        ft.loc[:,a0+m] = t
        m = m+1


    for j in [1, 3, 6]:
        a = 0
        k = datad[(datad['purchase_dt']<time_point)&
                      ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                      (datad['mrch_category_02'] == dictionary['TRAVEL AGENCIES'])
                     ]

        t = k.groupby('acct_num_share').count()['rub_amt']
        t[t>0] = 1
        t[t!=1] = 0
        ft.loc[:,a0+m] = t
        m = m+1



    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Средние тарты в категориях 
    #'TRAVEL AGENCIES'
    #за 1, 3, 6, 12 мес
    #столбцы номер 147, .. , 150

    a0 = 147
    a1 = 150
    m = 0

    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['TRAVEL AGENCIES'])
                 ]
        a = k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно

    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Средние тарты в категориях 
    #AIRLINES
    #за 1, 3, 6, 12 мес
    #столбцы номер 151, .. , 154

    a0 = 151
    a1 = 154
    m = 0

    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['AIRLINES'])
                 ]
        a = k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Средние тарты в категориях 
    #RESTAURANTS
    #за 1, 3, 6, 12 мес
    #столбцы номер 155, .. , 158

    a0 = 155
    a1 = 158
    m = 0

    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['RESTAURANTS'])
                 ]
        a = k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Средние тарты в категориях 
    #SPA / BEAUTY SERVICES
    #за 1, 3, 6, 12 мес
    #столбцы номер 159, .. , 162

    a0 = 159
    a1 = 162
    m = 0

    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['SPA / BEAUTY SERVICES'])
                 ]
        a = k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Средние тарты в категориях 
    #ENTERTAINMENT
    #за 1, 3, 6, 12 мес
    #столбцы номер 163, .. , 166

    a0 = 163
    a1 = 166
    m = 0

    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_03'] == dictionary['ENTERTAINMENT'])
                 ]
        a = k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Средние тарты в категориях 
    #LUXURY
    #за 1, 3, 6, 12 мес
    #столбцы номер 167, .. , 170

    a0 = 167
    a1 = 170
    m = 0

    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['LUXURY GOODS RETAIL'])
                 ]
        a = k.groupby('acct_num_share').mean()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)

    #Ускорено

    #Писал PHILIPP
    #Дисперсия трат в категориях 
    #TRAVEL AGENCIES
    #за 1, 3, 6, 12 мес
    #столбцы номер 171, .. , 174

    a0 = 171
    a1 = 174
    m = 0

    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['TRAVEL AGENCIES'])
                 ]
        a = k.groupby('acct_num_share').var()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно, но есть много нулей в начале, тк мало людей там покупали
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Дисперсия трат в категориях 
    #AIRLINES
    #за 1, 3, 6, 12 мес
    #столбцы номер 175, .. , 178

    a0 = 175
    a1 = 178

    m=0
    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['AIRLINES'])
                 ]
        a = k.groupby('acct_num_share').var()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 


    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Дисперсия трат в категориях 
    #RESTAURANTS
    #за 1, 3, 6, 12 мес
    #столбцы номер 179, .. , 182

    a0 = 179
    a1 = 182


    m=0
    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['RESTAURANTS'])
                 ]
        a = k.groupby('acct_num_share').var()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Дисперсия трат в категориях 
    #SPA / BEAUTY SERVICES
    #за 1, 3, 6, 12 мес
    #столбцы номер 183, .. , 186

    a0 = 183
    a1 = 186


    m=0
    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['SPA / BEAUTY SERVICES'])
                 ]
        a = k.groupby('acct_num_share').var()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 


    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Медианы тарт в категориях 
    #TRAVEL AGENCIES
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 234, .. , 238

    a0 = 234
    a1 = 238
    m=0

    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['TRAVEL AGENCIES'])
                 ]
        a = k.groupby('acct_num_share').median()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 
        #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Медиана тарт в категориях 
    #AIRLINES
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 239, .. , 243

    a0 = 239
    a1 = 243
    m = 0

    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['AIRLINES'])
                 ]
        a = k.groupby('acct_num_share').median()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Медиана тарты в категориях 
    #Restaurants
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 244, .. , 248
    a0 = 244
    a1 = 248
    m = 0

    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['RESTAURANTS'])
                 ]
        a = k.groupby('acct_num_share').median()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Медиана тарты в категориях 
    #SPA / BEAUTY SERVICES
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 249, .. , 253

    a0 = 249
    a1 = 253

    m=0
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['SPA / BEAUTY SERVICES'])
                 ]
        a = k.groupby('acct_num_share').median()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 


    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Median тарты в категориях 
    #ENTERTAIMENT
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 254, .. , 258

    a0 = 254
    a1 = 258
    m = 0

    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_03'] == dictionary['ENTERTAINMENT'])
                 ]
        a = k.groupby('acct_num_share').median()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Median тарты в категориях 
    #LUXURY
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 259, .. , 263
    a0 = 259
    a1 = 263

    m = 0

    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['LUXURY GOODS RETAIL'])
                 ]
        a = k.groupby('acct_num_share').median()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0) 

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Количество потраченных денег
    #Не в России
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 274, .. , 278
    a0 = 274
    a1 = 278


    m = 0
    for j in [1, 3, 6, 9, 12]:

        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])
                 ]

        a = k.groupby('acct_num_share').sum()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Количество потраченных денег
    #в России
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 274, .. , 278
    a0 = 289
    a1 = 293


    m = 0
    for j in [1, 3, 6, 9, 12]:

        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_country_nm'] == dictionary['RUSSIAN FEDERATION'])
                 ]

        a = k.groupby('acct_num_share').sum()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено

    #Писал PHILIPP
    #Самая большая трата
    #за 1, 3, 6, 9, 12 мес
    #столбцы номер 294, .. , 298
    a0 = 294
    a1 = 298


    m = 0
    for j in [1, 3, 6, 9, 12]:

        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])
                  ]

        a = k.groupby('acct_num_share').max()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
#ФИЧИ САНИ

       ###ПИСАЛ ШУРЛО
    #CONSTRUCTION SERVICES merch 02
    #SWIMMING POOLS/SALES/SERV merch 01
    #наличие трат в этих категориях
    #1м,3м,6м,12м
    #Столбцы номер 203-206

    a0 = 203
    a1 = 207


    m = 0
    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (datad['mrch_category_02'] == dictionary['CONSTRUCTION SERVICES'])&
                      (datad['mrch_category_01'] == dictionary['SWIMMING POOLS/SALES/SERV'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

        ft.loc[:,a0+m] = k
        ft[a0+m] = ft[a0+m].fillna(0)#  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
        ft[a0+m] = list(map(lambda x: 1 if x>0 else 0,ft[a0+m]))

        m +=1
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #LEISURE GOODS RETAIL merch 02
    #наличие трат в этих категориях
     #1м,3м,6м
    #Столбцы номер 210-212

    a0 = 210
    a1 = 213

    m = 0
    for j in [1, 3, 6]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (datad['mrch_category_02'] == dictionary['LEISURE GOODS RETAIL'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

        ft.loc[:,a0+m] = k
        ft[a0+m] = ft[a0+m].fillna(0)#  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
        ft[a0+m] = list(map(lambda x: 1 if x>0 else 0,ft[a0+m]))

        m +=1
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #INSURANCE SALES/UNDERWRITE or
    #INSURANCE PREMIUMS or
    #INSURANCE - DEFAULT
    #любые из трех
    #наличие трат 1м , 3 м
    #Столбцы номер 218-219

    a0 = 218
    a1 = 220


    m = 0
    for j in [1, 3]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (
                      (datad['mrch_category_01'] == dictionary['INSURANCE SALES/UNDERWRITE']) |
                      (datad['mrch_category_01'] == dictionary['INSURANCE SALES/UNDERWRITE']) |
                      (datad['mrch_category_01'] == dictionary['INSURANCE SALES/UNDERWRITE'])
                      )]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

        ft.loc[:,a0+m] = k
        ft[a0+m] = ft[a0+m].fillna(0)#  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
        ft[a0+m] = list(map(lambda x: 1 if x>0 else 0,ft[a0+m]))

        m = m + 1
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #DUTY FREE STORES
    #Проверяем наличие трат
    # 0|1
    #Столбец 207

    a0 = 207


    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['mrch_category_01'] == dictionary['DUTY FREE STORES'])]

    k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

    ft.loc[:,a0] = k
    ft[a0] = ft[a0].fillna(0)#  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
    ft[a0] = list(map(lambda x: 1 if x>0 else 0,ft[a0]))
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #ART DEALERS & GALLERIES
    #Проверяем наличие трат
    # 0|1
    #Столбец 208

    a0 = 208


    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['mrch_category_01'] == dictionary['ART DEALERS & GALLERIES'])]

    k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

    ft.loc[:,a0] = k
    ft[a0] = ft[a0].fillna(0)#  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
    ft[a0] = list(map(lambda x: 1 if x>0 else 0,ft[a0]))
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #POLITICAL ORGANIZATIONS
    #Проверяем наличие трат
    # 0|1
    #Столбец 213

    a0 = 213


    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['mrch_category_01'] == dictionary['POLITICAL ORGANIZATIONS'])]

    k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

    ft.loc[:,a0] = k
    ft[a0] = ft[a0].fillna(0)#  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
    ft[a0] = list(map(lambda x: 1 if x>0 else 0,ft[a0]))
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #RELIGIOUS ORGANIZATIONS
    #Проверяем наличие трат
    # 0|1
    #Столбец 214

    a0 = 214

    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['mrch_category_01'] == dictionary['RELIGIOUS ORGANIZATIONS'])]

    k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

    ft.loc[:,a0] = k
    ft[a0] = ft[a0].fillna(0)#  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
    ft[a0] = list(map(lambda x: 1 if x>0 else 0,ft[a0]))
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!          
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #TAX PAYMENTS
    #Проверяем траты за последний год и сравниваем с константами
    # числа для сравнения > 5000,10000,20000,30000,50000 Цифры меня смущают
    #Столбец 220-224

    a0 = 220
    a1 = 225


    m = 0
    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months = 12)) &
                      (datad['mrch_category_01'] == dictionary['TAX PAYMENTS'])]

    k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

    for j in [5000,10000,20000,30000,50000]:
        ft.loc[:,a0+m] = k
        ft[a0+m] = ft[a0+m].fillna(0)
        ft[a0+m] = list(map(lambda x: 1 if x>=j else 0,ft[a0+m]))
        m += 1
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!  
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #FINANCIAL SERVICES
    #Проверяем траты за последний год и сравниваем с константами
    # числа для сравнения > 10000,50000,100000 Цифры меня смущают
    #Столбец 225-227

    a0 = 225
    a1 = 228


    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months = 12)) &
                      (datad['mrch_category_02'] == dictionary['FINANCIAL SERVICES'])]


    k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

    m = 0
    for j in [10000,50000,100000]:
        ft.loc[:,a0+m] = k
        ft[a0+m] = ft[a0+m].fillna(0)
        ft[a0+m] = list(map(lambda x: 1 if x>=j else 0,ft[a0+m]))
        m += 1
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!        
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #Покупки(~выезд за границу) за границу в последние
    #Проверяем опкупки F2F за границей и ставим единичку, если они были (FACE-TO-FACE) (RUSSIAN FEDERATION)
    #За последние   1, 3, 6, 9, 12 мес
    #Столбец 228-232

    a0 = 228
    a1 = 233


    m = 0
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])&
                      (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

    #    k = map(lambda x: 1 if x>0 else 0,k)

        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0) #  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
        ft[m] = list(map(lambda x: 1 if x>0 else 0,ft[m]))
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #Дисперсия трат
    #TOURIST ATTRACTIONS AND XHBT 
    #За последние   1, 3, 6, 9, 12 мес
    #Столбец 198-202

    a0 = 198
    a1 = 203


    m = 0
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (datad['mrch_category_01'] == dictionary['TOURIST ATTRACTIONS AND XHBT'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').std()['rub_amt']

        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0) #  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #Дисперсия трат
    #LUXURY GOODS RETAIL 
    #За последние   1, 3, 6,  12 мес
    #Столбец 198-202

    a0 = 198
    a1 = 203


    m = 0
    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (datad['mrch_category_01'] == dictionary['LUXURY GOODS RETAIL'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').std()['rub_amt']

        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0) #  ?????????????????????????????????????

    pbar.update(1)

    ###ПИСАЛ ШУРЛО
    #Дисперсия трат
    #TRAVEL 
    #За последние   1, 3, 6,  12 мес
    #Столбец 187-190

    a0 = 187
    a1 = 191


    m = 0
    for j in [1, 3, 6, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (datad['mrch_category_01'] == dictionary['TRAVEL'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').std()['rub_amt']


        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0) #  ?????????????????????????????????????

    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
        ###ПИСАЛ ШУРЛО
    #Количество F2F покупок за границей
    #За последние   1, 3, 6,9, 12 мес
    #Столбец 264-268

    a0 = 264
    a1 = 269


    m = 0
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j)) &
                      (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').count()['rub_amt']


        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0)
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #Самая маленькая трата
    #За последние   1, 3, 6,9, 12 мес
    #Столбец 299-303

    a0 = 299
    a1 = 304


    m = 0
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j))]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').min()['rub_amt']


        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0) # ВОТ тут вопросик  ставить хз шо????????????????????
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #Сумма покупок в e-com
    #За последние   1, 3, 6,9, 12 мес
    #Столбец 304-308

    a0 = 304
    a1 = 309

    m = 0
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                      (datad['purchase_dt']>time_point-DateOffset(months=j))&
                      (datad['f2f_ecomm_flg'] == dictionary['E-COMM'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0)
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    
    ###ПИСАЛ ШУРЛО
    #Отношение трат конкретного чела, к среднему по всем клиентам
    #За последние   1, 3, 6,9, 12 мес
    #Столбец 309-313

    a0 = 309
    a1 = 314

    m = 0
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                  (datad['purchase_dt']>time_point-DateOffset(months=j))]

        avg_for_all_client = datad[(datad['purchase_dt']<time_point) &
                                                (datad['purchase_dt']>time_point-DateOffset(months=j))]

        avg_for_all_client = float(avg_for_all_client[['rub_amt','acct_num_share']].groupby('acct_num_share').std().std())

        a = k[['rub_amt','acct_num_share']].groupby('acct_num_share').std()/avg_for_all_client

        ft.loc[:,a0+m] = a
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0)
    pbar.update(1)
    
    ###ПИСАЛ ШУРЛО
    #Траты за границей
    #За последние   1, 3, 6, 9, 12 мес
    #Столбец 314-318

    a0 = 314
    a1 = 319


    m = 0    
    for j in [1, 3, 6, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point) &
                    (datad['purchase_dt']>time_point-DateOffset(months=j))&
                    (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION']) &
                    (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])]

        k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']

        ft.loc[:,a0+m] = k
        m +=1

    for m in range(a0,a1):
        ft[m] = ft[m].fillna(0)
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    ###ПИСАЛ ШУРЛО
    #Количество дней с последней поездки
    #Столбец 319

    a0 = 319

    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION']) &
                      (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])]

    k = k[['purchase_dt','acct_num_share']].groupby('acct_num_share').max()['purchase_dt']
    k = (time_point-k).dt.days

    ft.loc[:,a0] = k


    ft[a0] = ft[a0].fillna(1000) #  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
    #УЛУЧШЕНО!!!!!!!!!!!!!!!!!!
    pbar.update(1)
    
    
    ##############
    #СНОВА ФИЛИПП#
    ##############
    
    #Писал PHILIPP
    #Фичи из столбцов 320,... , 327
    #summ_m*
    a0 = 320
    a1 = 327

    m = 0

    for j in [2, 3, 4, 5, 6, 10, 11, 12]:
        
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])]

        t = k.groupby('acct_num_share').sum()['rub_amt']

        ft.loc[:,a0+m] = t
        m=m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)

    #Код работает верно
    pbar.update(1)
    #Ускорено


    #Писал PHILIPP
    #RESTAURANTS_amt_f2f_purch_9mes
    #за 9 мес
    #столбцы номер 328

    a0 = 328
    a1 = 328
    m = 0

    for j in [9]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['RESTAURANTS'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])
                 ]
        a = k.groupby('acct_num_share').count()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено


    #Писал PHILIPP
    #RESTAURANTS_sum_f2f_purch_9mes
    #за 9 мес
    #столбцы номер 329

    a0 = 329
    a1 = 329
    m = 0

    for j in [9]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['RESTAURANTS'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])
                 ]
        a = k.groupby('acct_num_share').sum()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    

    #Писал PHILIPP
    #RESTAURANTS_sum_home_purch_*mes
    #за 1,9,12 мес
    #столбцы номер 330-332

    a0 = 330
    a1 = 332
    m = 0

    for j in [1, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['RESTAURANTS'])&
                 (datad['mrch_country_nm'] == dictionary['RUSSIAN FEDERATION'])
                 ]
        a = k.groupby('acct_num_share').sum()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    

    #Писал PHILIPP
    #SUPERMARKETS_amt_f2f_purch_9mes
    #за 9 мес
    #столбцы номер 334

    a0 = 334
    a1 = 334
    m = 0

    for j in [9]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['SUPERMARKETS'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])
                 ]
        a = k.groupby('acct_num_share').count()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    

    #Писал PHILIPP
    #SUPERMARKETS_sum_f2f_purch_9mes
    #за 9 мес
    #столбцы номер 335

    a0 = 335
    a1 = 335
    m = 0

    for j in [9]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['SUPERMARKETS'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])
                 ]
        a = k.groupby('acct_num_share').sum()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    

    #Писал PHILIPP
    #SUPERMARKETS_sum_home_purch_*mes
    #за 1,9,12 мес
    #столбцы номер 336-338

    a0 = 336
    a1 = 338
    m = 0

    for j in [1, 9, 12]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['SUPERMARKETS'])&
                 (datad['mrch_country_nm'] == dictionary['RUSSIAN FEDERATION'])
                 ]
        a = k.groupby('acct_num_share').sum()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    

    #Писал PHILIPP
    #TRANSPORTATION_sum_home_purch_1mes
    #за 1 мес
    #столбцы номер 339

    a0 = 339
    a1 = 339
    m = 0

    for j in [1]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['TRANSPORTATION'])&
                 (datad['mrch_country_nm'] == dictionary['RUSSIAN FEDERATION'])
                 ]
        a = k.groupby('acct_num_share').sum()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    

    #Писал PHILIPP
    #FASHION RETAIL_amt_f2f_purch_9mes
    #за 9 мес
    #столбцы номер 340

    a0 = 340
    a1 = 340
    m = 0

    for j in [9]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['FASHION RETAIL'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])
                 ]
        a = k.groupby('acct_num_share').count()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    

    #Писал PHILIPP
    #FOOD RETAIL_amt_f2f_purch_9mes
    #за 9 мес
    #столбцы номер 341

    a0 = 341
    a1 = 341
    m = 0

    for j in [9]:
        k = datad[(datad['purchase_dt']<time_point)&
                  ( time_point - DateOffset(months=j) <datad['purchase_dt'])&
                  (datad['mrch_category_02'] == dictionary['FOOD RETAIL'])&
                  (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])
                 ]
        a = k.groupby('acct_num_share').count()['rub_amt']
        ft.loc[:,a0+m] = a
        m = m+1

    for m in range(a0,a1+1):
        ft[m] = ft[m].fillna(0)
        
    #Код работает верно
    pbar.update(1)
    #Ускорено
    
    ###ПИСАЛ ШУРЛО
    #Количество дней в поездках
    #Столбец 342

    a0 = 342

    k = datad[(datad['purchase_dt']<time_point) &
                      (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION']) &
                      (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])]

    k = k[['purchase_dt','acct_num_share']].groupby('acct_num_share').nunique()['purchase_dt']

    ft.loc[:,a0] = k


    ft[a0] = ft[a0].fillna(0) #  Решил заполнять нулями потому что ноль, это значит их нет ухахах!
    pbar.update(1)
    #Ускорено
    
    

    ####################
        
    file_output = 'ft_' + str(mth) + '.csv'
    ft.to_csv(file_output)
    
    cat_per = pd.DataFrame(np.zeros((indd.shape[0],1)), index=indd)
    time_spamp1 = time_point + DateOffset(months=1)
    time_spamp2 = time_point + DateOffset(months=3)
    
    
    k = datad[(datad['purchase_dt']>time_spamp1)&
                     (datad['purchase_dt']<time_spamp2)&
                     (datad['f2f_ecomm_flg'] == dictionary['FACE-TO-FACE'])&
                     (datad['mrch_country_nm'] != dictionary['RUSSIAN FEDERATION'])]
    
    k = k[['rub_amt','acct_num_share']].groupby('acct_num_share').sum()['rub_amt']
    cat_per.loc[:, 0]=k
    cat_per[0]=cat_per[0].fillna(0)
    cat_per[0] = list(map(lambda x: 1 if x>0 else 0,cat_per[0])) 
                
    file_output_categ = 'categ_' + str(mth) + '.csv'
    cat_per.to_csv(file_output_categ)
    
    pbar.update(1)
    
    indd_output = 'indd_' + str(mth) + '.csv'
    np.savetxt(indd_output, indd, delimiter=',', fmt='%d')
    
    break
    
pbar.close()

100%|██████████| 81/81 [4:21:03<00:00, 193.38s/it]    


In [11]:
time_end = datetime.datetime.utcnow()

delta = time_end - time_start

# print('Number of cards: ', number_of_cards)
print('Requerd time: ' ,delta.total_seconds()/60, ' minutes')

NameError: name 'number_of_cards' is not defined

<b><i> Теперь все фичи, таргеты, а также номера карт, которые мы использовали находятся в 3 разных файлах в папке, в которой находится программа

In [12]:
ft

,0,1,2,3,4,5,6,7,8,9,...,333,334,335,336,337,338,339,340,341,342
1,23.000000,197.000000,741.375000,940.000000,1027.013794,1873.418823,2063.810059,2486.031982,0.000000,0.0,...,0.0,127.0,156009.0,5936.0,150232.0,241434.0,0.0,11.0,87.0,14.0
2,163.500000,445.333344,312.558838,481.842102,788.875732,1060.851196,1185.246216,1374.573853,0.000000,0.0,...,0.0,174.0,354622.0,20652.0,334912.0,473134.0,0.0,18.0,8.0,19.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,758.211548,688.491089,0.000000,0.0,...,0.0,36.0,19336.0,0.0,19336.0,50312.0,0.0,0.0,40.0,1.0
4,1119.800049,864.000000,864.000000,975.919983,1091.457153,1469.878540,1540.588623,2177.689453,0.000000,0.0,...,0.0,142.0,260462.0,30648.0,260462.0,304994.0,0.0,12.0,36.0,0.0
5,0.000000,1011.000000,864.333313,3106.315674,5164.568848,3293.008301,3169.458008,2963.847656,0.000000,0.0,...,0.0,178.0,135248.0,6008.0,131890.0,167442.0,4782.0,20.0,6.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125713,1235.291626,1804.659058,1846.083374,1946.444458,2362.389160,1958.500000,1775.536133,1650.511719,608.333313,1288.5,...,0.0,133.0,423198.0,8110.0,337961.0,417081.0,1682.0,18.0,33.0,58.0
125714,470.000000,624.055542,1924.702148,1645.875000,1388.642090,925.668213,967.725159,904.964600,287.000000,235.5,...,0.0,147.0,83399.0,13466.0,82759.0,97795.0,471.0,23.0,15.0,7.0
125715,0.000000,0.000000,0.000000,0.000000,0.000000,807.375000,1372.596313,1030.668579,0.000000,0.0,...,0.0,37.0,20301.0,0.0,17128.0,26556.0,0.0,1.0,0.0,16.0
125716,0.000000,0.000000,0.000000,0.000000,0.000000,652.761902,914.815491,998.954041,0.000000,0.0,...,0.0,48.0,29100.0,0.0,28857.0,49712.0,0.0,7.0,4.0,18.0


<hr>